<a href="https://colab.research.google.com/github/crybot/NapoleonZero/blob/master/NapoleonZero.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports


In [152]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
  # !pip install tensorflow-gpu==2.1.0
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
import numpy as np
import pandas as pd
import io

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Mon Oct 12 13:25:40 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    38W / 300W |  15619MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Install and log into Weights & Biases

# Dataset management


### Download dataset from GitHub


In [153]:
import subprocess
import os

cmd = ['git', 'clone', 'https://github.com/crybot/NapoleonZero.git']
subprocess.run(cmd)

DRIVE_PATH = './NapoleonZero'
# Read dataset file
with open('{path}/converted-depth1.csv'.format(path=DRIVE_PATH), 'r') as f:
  # use dtype=str when parsing the csv to avoid converting bitboards 
  # in integer values (e.g ....001 -> 1)
  df = pd.read_csv(f, header=None, dtype=str)
  dataset = np.array(df.values)

def string_to_matrix(bitboard):
  return(np.array(list(bitboard), dtype=np.float32).reshape(8,8))

print('Initial dataset shape')
print(dataset.shape)

np.random.seed(42) # Reset numpy's random seed
np.random.shuffle(dataset)
#dataset = dataset[:10000, :]

# Scores (target) preprocessing
scores = dataset[:, 16:].astype(np.float32) # score in centipawns
scores /= 100 # score is in centipawns, so we divide by 100 (i.e. pawn = 1)
dataset = dataset[np.abs(scores[:, 0]) < 10] # remove positions with too high scores
scores = dataset[:, 16:].astype(np.float32) # also remove the scores (see above)

colors = dataset[:, 12:13].astype(np.float32) # side to move: 0 white, 1 black
epsquares = dataset[:, 13:14].astype(np.float32) # enpassant square: 0-63, 65 if there is none
castlings = dataset[:, 14:15].astype(np.float32) # castling status: integer value
depths = dataset[:, 15:16].astype(np.float32) # depth of the search: 1-100
scores[colors == 1] *= -1.0 # change black's score perspective: always evaluate white's position


# The first 12 fields of the dataset are bitboards
bitboards = dataset[:, 0:12] 
# The bitboards dataset has shape Nx12X1 where 1 is a string of 64 characters.
# We want Nx12x8x8 floats
bitboards = np.array([[string_to_matrix(b) for b in bs] for bs in bitboards])

print(bitboards.shape)
print(scores.shape)
print(scores[1])
# reshaped = np.concatenate((bitboards, scores), axis=1)
# print(reshaped.shape)
# print(reshaped[0])


Initial dataset shape
(100000, 17)
(99793, 12, 8, 8)
(99793, 1)
[25.]


### Dataset splitting 

In [154]:
# ONLY USED TO LOG THE CONFIGURATION OF THE TRAINING STAGE INTO W&B
TRAIN_SPLIT = 0.7
VAL_SPLIT = 0.15
TEST_SPLIT = 0.15

# Split a given dataset into training, validation and test sets
def split_dataset(ds, train=0.7, val=0.15):
  train_ds = ds[0:int(len(dataset)*train)]
  val_ds = ds[int(len(dataset)*train):int(len(dataset)*(train + val))]
  test_ds = ds[int(len(dataset)*(train + val)):]
  return (train_ds, val_ds, test_ds)

(train_ds, val_ds, test_ds) = split_dataset(bitboards, TRAIN_SPLIT, VAL_SPLIT)
(train_color, val_color, test_color) = split_dataset(colors, TRAIN_SPLIT, VAL_SPLIT)
(train_y, val_y, test_y) = split_dataset(scores, TRAIN_SPLIT, VAL_SPLIT)

print(train_ds.shape)
print(val_ds.shape)
print(test_ds.shape)

print(train_y.shape)
print(val_y.shape)
print(test_y.shape)

(69855, 12, 8, 8)
(14969, 12, 8, 8)
(14969, 12, 8, 8)
(69855, 1)
(14969, 1)
(14969, 1)


# Model creation

## Configuration and Hyperparameters

In [155]:

# initial hyperparameters values
default_config = {
    # FIXED
    "train_split": TRAIN_SPLIT,
    "val_split": VAL_SPLIT,
    "test_split": TEST_SPLIT,
    "monitor": 'val_loss',
    "patience": 200,
    "batch_normalization": True,
    "seed": 42,

    # VARIABLE
    "epochs": 1000,
    "batch_size": 12000,
    "learning_rate": 1e-3,
    "weight_decay": 5e-5,
    "dropout_rate": 0.5,
    "cnn_kernels": 64,
    "cnn_window": (2,2),
    "cnn_layers": 3,
    "optimizer": 'adam',
    "activation": 'relu',
    "loss": 'mse'
}

val_tf_ds = tf.data.Dataset.from_tensor_slices(
    ({"bitboards_in": val_ds, "color_in": val_color},
    {'output': val_y})
    )
val_tf_ds = val_tf_ds.cache()
val_tf_ds = val_tf_ds.batch(len(val_ds)) # single batch
val_tf_ds = val_tf_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

print(val_tf_ds)
print(tf.__version__)

# print(list(val_tf_ds.as_numpy_iterator())[0][1]['output'])


<PrefetchDataset shapes: ({bitboards_in: (None, 12, 8, 8), color_in: (None, 1)}, {output: (None, 1)}), types: ({bitboards_in: tf.float32, color_in: tf.float32}, {output: tf.float32})>
2.3.0


## Convolutional/Connected model with skip connections


In [156]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Conv2D, Dropout, GaussianDropout, SeparableConv1D
from tensorflow.keras.layers import BatchNormalization, MaxPool2D
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, SGD, Adam, Nadam
from tensorflow.keras.losses import Huber

import tensorflow_datasets as tfds
import datetime
  
config = default_config

# TODO: try different orders of conv,bn,relu
def conv_block(input, filters=16, kernel_size=(2,2), activation='relu',
               input_shape=None,
               normalize=True,
               pool=False,
               decay=1e-4,
               data_format='channels_first'):
  if input_shape is not None:
    conv = layers.Conv2D(filters, kernel_size,
                        data_format=data_format,
                        padding='same',
                        kernel_regularizer=l2(decay),
                        input_shape=input_shape)(input)
  else:
    conv = layers.Conv2D(filters, kernel_size,
                        data_format=data_format,
                        padding='same',
                        kernel_regularizer=l2(decay))(input)

                       
  if activation is not None:
    conv = Activation(activation)(conv)
  if normalize:
    conv = layers.BatchNormalization()(conv)
  if pool:
    conv = MaxPool2D(pool_size=(2,2), data_format=data_format)(conv)

  return conv


def build_model(config):
  # Define inputs to the network
  main_input = Input(shape=(12, 8, 8), name='bitboards_in')
  color_input = Input(shape=(1,), name='color_in')

  # Boolean indicating whether we should use batch normalization or not
  normalize = config['batch_normalization'] 
  # Number of kernels (filters) for each layer of the CNN
  kernels = config['cnn_kernels']
  # Window size of each kernel
  window = config['cnn_window']
  decay = config['weight_decay']
  activation = config['activation']
  CNN_layers = config['cnn_layers']
  
  ################################################################################
  ########################## CONVOLUTIONAL LAYERS ################################
  # Stack of convolutional and pooling layers: each conv layer produces N filters
  # from its inputs with a window size of W. The output of the convolution 
  # is zero-padded to maintain the same dimensionality
  
  # First block
  conv = conv_block(main_input, kernels, window,
                    decay=decay,
                    activation=activation,
                    input_shape=(12,8,8))

  # Second block
  conv = conv_block(conv, kernels, window, activation, decay=decay, pool=True)
  # Third block
  kernels *= 2
  conv = conv_block(conv, kernels, window, activation, decay=decay, pool=True)
  # Fourth block
  kernels *= 2
  conv = conv_block(conv, kernels, window, activation, decay=decay, pool=False)

  # Series of non pooling convolutional blocks
  for i in range(CNN_layers):
    conv = conv_block(conv, kernels, window, activation, decay=decay)

  # Last CNN block
  kernels *= 2
  conv = conv_block(conv, kernels, window, activation, decay=decay)
  # Global pooling layer
  # conv = layers.GlobalMaxPool2D(data_format='channels_first')(conv)
  
  ################################################################################
  ################################################################################
  
  flattened = layers.Flatten()(conv)
  flattened = layers.Dropout(config['dropout_rate'])(flattened) # Apply dropout to approximate model set averaging
  flattened = layers.concatenate([flattened, color_input])

  
  ################################################################################
  ############################## DENSE LAYERS ####################################
  layer2 = Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(decay))(flattened)
  layer2 = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(decay))(layer2)
  # layer2 = Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(decay))(layer2)
  output = Dense(1, activation='linear', kernel_regularizer=tf.keras.regularizers.l2(decay),
                 name='output')(layer2)
  ################################################################################
  
  model = Model(inputs=[main_input, color_input],
                outputs=[output])

  return model

def get_optimizer(config):
  name = config['optimizer']
  if name =='sgd':
    return SGD(lr=config['learning_rate'], decay=config['weight_decay'], momentum=0.8, nesterov=False)
  elif name =='rmsprop':
    return RMSprop(lr=['config.learning_rate'], decay=1e-5)
  elif name =='adam':
    return Adam(lr=config['learning_rate'], beta_1=0.9, beta_2=0.999)
    # return Adam(lr=config.learning_rate, beta_1=0.9, beta_2=0.999, clipnorm=1.0)
  elif name =='nadam':
    return Nadam(lr=config['learning_rate'], beta_1=0.9, beta_2=0.999, clipnorm=1.0)
  return None

def train():
  tf_ds = tf.data.Dataset.from_tensor_slices(
      ({"bitboards_in": train_ds, "color_in": train_color},
      {'output': train_y}))
  # tf_ds = tf.data.Dataset.from_tensor_slices((train_ds, train_y))
  tf_ds = tf_ds.cache()
  tf_ds = tf_ds.batch(config['batch_size'], drop_remainder=False)
  tf_ds = tf_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  print(np.array(tf_ds.as_numpy_iterator()).shape)
  print(tf_ds)

  loss = config['loss']
  model = build_model(config)
  optimizer = get_optimizer(config)
  model.compile(optimizer=optimizer,
              loss=loss,
              metrics=[tf.metrics.RootMeanSquaredError()])
  model.summary()

  early_stopping = tf.keras.callbacks.EarlyStopping(
     monitor=config['monitor'],
     patience=config['patience'],
     min_delta=0.01,
     restore_best_weights=True,
     verbose=2)

  reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
     monitor='val_loss',
     factor=0.7,
     mode='min',
     verbose=1,
     patience=40, min_lr=0.0001)

  model.fit(tf_ds,
            epochs=config['epochs'], 
            # batch_size=config['batch_size'],
            validation_data=val_tf_ds,
            # validation_data=(val_dataset, val_target),
            verbose=2,
            callbacks=[
                      # early_stopping,
                      reduce_lr
                       ])

# Training

In [ ]:
train()

()
<PrefetchDataset shapes: ({bitboards_in: (None, 12, 8, 8), color_in: (None, 1)}, {output: (None, 1)}), types: ({bitboards_in: tf.float32, color_in: tf.float32}, {output: tf.float32})>
Model: "functional_45"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bitboards_in (InputLayer)       [(None, 12, 8, 8)]   0                                            
__________________________________________________________________________________________________
conv2d_176 (Conv2D)             (None, 64, 8, 8)     3136        bitboards_in[0][0]               
__________________________________________________________________________________________________
activation_176 (Activation)     (None, 64, 8, 8)     0           conv2d_176[0][0]                 
_________________________________________________________________________________________________